In [11]:
#Download the required data that we will use 
!wget http://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/sst.mon.mean.trefadj.anom.1880to2018.nc
!wget http://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/nino34.long.anom.data.txt
!wget https://downloads.psl.noaa.gov/Datasets/COBE/sst.mon.mean.nc

--2023-06-08 09:54:46--  http://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/sst.mon.mean.trefadj.anom.1880to2018.nc
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.107, 128.55.206.112, 128.55.206.113, ...
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.107|:80... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/sst.mon.mean.trefadj.anom.1880to2018.nc [following]
--2023-06-08 09:54:47--  https://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/sst.mon.mean.trefadj.anom.1880to2018.nc
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432481041 (412M) [application/x-netcdf]
Saving to: ‘sst.mon.mean.trefadj.anom.1880to2018.nc’

sst.mon.mean.trefad 100%[===================>] 412.45M  2.74MB/s    in 1m 47s  

2023-06-08 09:56:34 (3.87 MB/s) - ‘sst.mon.mean.trefadj.anom.1880to2018.nc’ s

In [12]:
!wget https://downloads.psl.noaa.gov/Datasets/COBE/sst.mon.mean.nc

--2023-06-08 09:57:37--  https://downloads.psl.noaa.gov/Datasets/COBE/sst.mon.mean.nc
Resolving downloads.psl.noaa.gov (downloads.psl.noaa.gov)... 140.172.38.86
Connecting to downloads.psl.noaa.gov (downloads.psl.noaa.gov)|140.172.38.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164523484 (157M) [application/x-netcdf]
Saving to: ‘sst.mon.mean.nc’

sst.mon.mean.nc     100%[===================>] 156.90M  2.94MB/s    in 66s     

2023-06-08 09:58:43 (2.39 MB/s) - ‘sst.mon.mean.nc’ saved [164523484/164523484]



# Import Necessary Libraries

In [34]:
# Principal Libraries 
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

#Dashboard Libraries

import panel as pn
import hvplot.pandas
import holoviews as hv
pn.extension('tabulator')
pn.extension('plotly')

# Mapper Libraries
import kmapper as km
from kmapper.jupyter import display
import umap.umap_ as umap #Uniform Manifold Approximation and Projection for Dimension Reduction
import sklearn.manifold as manifold #https://scikit-learn.org/stable/modules/manifold.html
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Periodicity and Homology Libraries
from ripser import ripser, Rips
import persim
from sklearn.metrics import pairwise_distances
import gudhi as gd
from gtda.time_series import SingleTakensEmbedding, takens_embedding_optimal_parameters
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram, plot_point_cloud

# Global Projections Libraries
import xarray as xr, cartopy.crs as crs
import hvplot.xarray  # noqa

# sorry about this
import warnings
warnings.filterwarnings("ignore")

# Forecasting
import scipy.stats
from sklearn.metrics import mean_squared_error
from gtda.diagrams import Amplitude
from gtda.time_series import TakensEmbedding
from gtda.homology import VietorisRipsPersistence
from sklearn.ensemble import RandomForestRegressor
from gtda.pipeline import make_pipeline
import xgboost as xgb


# Get Data and Data Wrangling

In [2]:
def get_anomalies_data_ds(data, var_name, start_date, end_date):  
    '''
    - Open the original SST dataset.
    - Select the time period from 1991 to 2020 to create a new dataset.
    - Calculate the long-term mean (LTM) of the SST for each month over this period.
    - Calculate the anomalies for each month by subtracting the LTM from the SST over the same period.
    - Add the LTM and anomalies as new variables to the dataset for the period 1991-2020.
    '''
    # Assign the data
    ds = data
    # Select the period start_date - end_date
    new_data = ds.sel(time=slice(start_date, end_date))

    # Calculate the long-term mean (LTM) for each month over the period start_date - end_date
    ltm = new_data['sst'].groupby('time.month').mean('time')

    # Calculate the anomalies for each month by subtracting the LTM from the SST
    sst_anomalies = new_data['sst'].groupby('time.month') - ltm

    # Add the LTM and the anomalies to the original dataset as new variables
    new_data['ltm'] = ltm
    new_data['anom'] = sst_anomalies

    # Now, the dataset ds has three variables: 'sst', 'ltm', and 'sst_anomalies'
    return new_data

def get_complete_anomalies_ds():
    ds = xr.open_dataset('sst.mon.mean.nc')
    ds_1991_2023 = get_anomalies_data_ds(ds, var_name = 'sst', start_date = '1991', end_date = '2023')
    anom_ds = xr.open_dataset('sst.mon.mean.trefadj.anom.1880to2018.nc')
    
    # Select 'anom' variable from ds_1991_2020 dataset and 'sst' from anom dataset
    ds_2019_2023_anom = ds_1991_2023.sel(time=slice('2019', '2023'))['anom']
    anom_1880_2018_sst = anom_ds['sst']
    
    # Make sure that both dataarrays have the same name to avoid conflict while concatenating
    ds_2019_2023_anom.name = 'sst'
    anom_1880_2018_sst.name = 'sst'
    
    # Convert DataArrays to Dataset
    ds_2019_2023_anom = ds_2019_2023_anom.to_dataset()
    anom_1880_2018_sst = anom_1880_2018_sst.to_dataset()
    
    # Merge the two Datasets
    anomalies_1880to2023 = xr.merge([anom_1880_2018_sst, ds_2019_2023_anom])
    return anomalies_1880to2023

def get_forecasting_data(tda = False):
    ds = get_complete_anomalies_ds()
    sst = ds['sst']
    if tda:
        sst = sst.fillna(0)
        X = sst
    else:
        num_time_steps = sst.shape[0]
        #sst is a 3D array: (time_steps, lat, lon)
        #in this tutorial, we will not be using ML models that take
        #advantage of the spatial nature of global temperature
        #therefore, we reshape sst into a 2D array: (time_steps, lat*lon)
        #(At each time step, there are lat*lon predictors)
        sst = sst.values.reshape(num_time_steps, -1)
        sst[np.isnan(sst)] = 0
        X = sst

    _, anoms = get_sst_anoms()
    y = anoms.query('DATE >= "1880" & DATE < "2023-04-01"')

    # Assuming your DataFrame is named 'df'
    y['DATE'] = pd.to_datetime(y['DATE'])  # Ensure 'DATE' column is in datetime format
    y.set_index('DATE', inplace=True)  # Set 'DATE' as the index

    # Convert 'ANOM' column to pandas Series
    y = y['ANOM']
    y.index = pd.to_datetime(y.index)
    y.index.name = None
    return X, y


In [3]:
def convert_to_numeric(df):
  for col in df.columns:
      if col != 'DATE':
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            pass
  return df

def long_data(df, type = 'ANOM'):
    # Melt the dataframe to long format
    df_long = df.melt(id_vars='YEAR', var_name='MONTH', value_name=type)
    # Extract the month number from the month column
    df_long['MONTH'] = pd.to_datetime(df_long['MONTH'], format='%b').dt.month
    # Create a datetime index
    df_long['DATE'] = pd.to_datetime(df_long[['YEAR', 'MONTH']].assign(DAY=1))
    # Drop the YEAR and MONTH columns
    df_long = df_long.drop(columns=['YEAR', 'MONTH'])
    # Sort Values by DATE
    df_long = df_long.sort_values(by='DATE')
    df_long = df_long.reset_index(drop=True)
    return df_long[['DATE', type]]
    

def get_sst_data_comb():
    comb = pd.read_fwf('https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt', colspecs="infer")
    # Group by 'Year' and add a sequential month number starting from 1
    comb['MONTH'] = comb.groupby('YR').cumcount() + 1
    # add a leading zero to the month number if it is less than 2 digits
    comb['MONTH'] = comb['MONTH'].apply(lambda x: '{0:0>2}'.format(x))
    # create a new column 'DATE' by concatenating the 'YR' and 'MONTH' columns
    comb['DATE'] = comb['YR'].astype(str) + '-' + comb['MONTH'].astype(str)
    # drop the 'YR' and 'MONTH' columns and convert the 'DATE' column to datetime format
    comb['DATE'] = pd.to_datetime(comb['DATE'], format='%Y-%m')

    sst_data = comb[['DATE', 'TOTAL']].copy()
    sst_anom = comb[['DATE', 'ANOM']].copy()

    sst_data.rename(columns = {'TOTAL':'TEMP'}, inplace = True)

    return sst_data, sst_anom


def data_by_season(data, col):
    df = data.copy()
    # First, we create two new columns: 'YEAR' and 'SEASON'
    df['YEAR'] = df['DATE'].dt.year
    df['MONTH'] = df['DATE'].dt.month.map({
        1: 'JAN', 2: 'FEB', 3: 'MAR', 4: 'APR', 5: 'MAY', 
        6: 'JUN', 7: 'JUL', 8: 'AUG', 9: 'SEP', 10: 'OCT', 
        11: 'NOV', 12: 'DEC'
    })
    # Then, we pivot the DataFrame to get one row per year and one column per season
    pivot_df = df.pivot(index='YEAR', columns='MONTH', values=col)

    # If you want to reorder the columns
    ordered_columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG','SEP', 'OCT', 'NOV', 'DEC']
    pivot_df = pivot_df[ordered_columns]

    pivot_df.columns.name = None

    df_reset = pivot_df.reset_index().rename(columns={"index": "YEAR"})
    return df_reset


def get_sst_data():
    sst_data_long , _ = get_sst_data_comb()
    compact_sst = data_by_season(sst_data_long, 'TEMP')
    return compact_sst.drop(compact_sst.tail(1).index), sst_data_long

def get_sst_anoms():    
    anom1 = pd.read_fwf('nino34.long.anom.data.txt', colspecs="infer", header=None)
    anom1.columns = ['YEAR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

    # Melt the dataframe to long format
    df_long = anom1.melt(id_vars='YEAR', var_name='MONTH', value_name='ANOM')

    # Extract the month number from the month column
    df_long['MONTH'] = pd.to_datetime(df_long['MONTH'], format='%b').dt.month

    # Create a datetime index
    df_long['DATE'] = pd.to_datetime(df_long[['YEAR', 'MONTH']].assign(DAY=1))

    # Drop the YEAR and MONTH columns
    df_long = df_long.drop(columns=['YEAR', 'MONTH'])

    # Sort Values by DATE
    df_long = df_long.sort_values(by='DATE')

    df_long = df_long.query('DATE < "2019"')

    nino34_df = df_long[['DATE', 'ANOM']].reset_index(drop=True)

    _, anom_long = get_sst_data_comb()

    sst_anom_long = pd.concat([nino34_df, anom_long.query('(DATE >= "2019")')], ignore_index=True)

    compact_anom = data_by_season(sst_anom_long, 'ANOM')

    return compact_anom.drop(compact_anom.tail(1).index), sst_anom_long # drop last row beacause 2023 is not a complete year

def get_olr_data():
    olr_data = pd.read_csv('https://www.cpc.ncep.noaa.gov/data/indices/olr', skiprows = 3, sep="\s+", nrows = 49).dropna()
    olr_data = convert_to_numeric(olr_data)
    return olr_data, long_data(olr_data, 'OLR')

def get_olr_anoms():
    olr_anom = pd.read_csv('https://www.cpc.ncep.noaa.gov/data/indices/olr', skiprows = 110, sep="\s+", nrows = 49).dropna()
    olr_anom = convert_to_numeric(olr_anom)
    return olr_anom, long_data(olr_anom, 'ANOM')

def get_soi_anoms():
    soi_data = pd.read_csv('https://www.cpc.ncep.noaa.gov/data/indices/soi', skiprows = 87, delimiter=r"\s+", nrows = 72)
    return soi_data, long_data(soi_data, 'ANOM')

def get_merged_data():    
    _, sst_anom_long = get_sst_anoms()
    _, sst_data_long = get_sst_data()
    _, olr_data_long = get_olr_data()
    _, olr_anom_long = get_olr_anoms()
    _, soi_anom_long = get_soi_anoms()

    sst_anom_long.rename(columns={'ANOM': 'SST ANOM'}, inplace=True)
    olr_anom_long.rename(columns={'ANOM': 'OLR ANOM'}, inplace=True)
    soi_anom_long.rename(columns={'ANOM': 'SOI ANOM'}, inplace=True)

    df1 = pd.merge(sst_anom_long, sst_data_long, on='DATE', how='outer')
    df2 = pd.merge(df1, olr_data_long, on='DATE', how='outer')
    df3 = pd.merge(df2, olr_anom_long, on='DATE', how='outer')
    merged_data = pd.merge(df3, soi_anom_long, on='DATE', how='outer')
    return merged_data



# Plots for measures

In [4]:
comb = get_merged_data()
comb['YEAR'] = comb['DATE'].dt.year

# Make DataFrame Pipeline Interactive
idf_merged = comb.interactive()

start_year = comb[['DATE','TEMP']].dropna().DATE.dt.year.min()
end_year = comb[['DATE','TEMP']].dropna().DATE.dt.year.max()

year_slider_start_comb = pn.widgets.IntSlider(
                                      name='Year Slider Start', 
                                      start=start_year, 
                                      end=end_year, 
                                      step=1, 
                                      value=1990
                                    )

year_slider_end_comb = pn.widgets.IntSlider(
                                    name='Year Slider End', 
                                    start=start_year, 
                                    end=end_year, step=1, 
                                    value=2020
                                    )

plot_measures_button = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    button_type = 'primary',
    options= list(['TEMP','OLR'])
)

In [5]:
measures_pipeline = (
    idf_merged[
        ['DATE', plot_measures_button]
        ][
          (idf_merged.YEAR >= year_slider_start_comb) &
          (idf_merged.YEAR <= year_slider_end_comb)
         ]
    .reset_index()
    .sort_values(by='DATE')  
    .reset_index(drop=True)
    .dropna()
)

In [6]:
measures_plot = measures_pipeline.hvplot(x = 'DATE', y= plot_measures_button,line_width=2, color = 'SlateGray')

# Plots for anomalies

In [7]:
start_year1 = comb.DATE.dt.year.min()
end_year1 = comb.DATE.dt.year.max()

year_slider_start_anom = pn.widgets.IntSlider(
                                name='Year Slider Start', 
                                start=start_year1, 
                                end=end_year1, step=1,
                                value=1990
                                )

year_slider_end_anom = pn.widgets.IntSlider(
                                    name='Year Slider End', 
                                    start=start_year1,
                                    end=end_year1, step=1,
                                    value=2020
                                    )

plot_anom_button = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    button_type = 'primary',
    options= list(['SST ANOM','OLR ANOM', 'SOI ANOM'])
)

In [8]:
anom_pipeline = (
    idf_merged[
        ['DATE', plot_anom_button]
        ][
          (idf_merged.YEAR >= year_slider_start_anom) &
          (idf_merged.YEAR <= year_slider_end_anom)
         ]
    .reset_index()
    .sort_values(by='DATE')  
    .reset_index(drop=True)
    .dropna()
)

In [9]:
from bokeh.models import CustomJSTickFormatter

formatter = CustomJSTickFormatter(code="return '';")

bar_plot = anom_pipeline.hvplot.bar(
    x = 'DATE', 
    y = plot_anom_button, 
    color=plot_anom_button, 
    colorbar=True, 
    clabel="ANOMALIES", 
    cmap="coolwarm",
    width=1500
).opts(xformatter=formatter)


red_line = hv.HLine(0.5).opts(color='red', line_width=2)
blue_line = hv.HLine(-0.5).opts(color='blue', line_width=2)

anom_plot = bar_plot * red_line * blue_line

# Geographic Projections

In [10]:
anom_ds = get_complete_anomalies_ds()
temp_ds = xr.open_dataset('sst.mon.mean.nc')

longitude = slice(120, 290)

anom_ds = anom_ds.sel(lon=longitude)
temp_ds = temp_ds.sel(lon=longitude)

# anom_ds['sst'] = anom_ds['sst'].fillna(0)  # Fill missing values with zeros

proj = crs.Orthographic(-155, 0)

anom_globe = anom_ds.sst.interactive.sel(time=pn.widgets.DiscreteSlider).hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True, 
    cmap='coolwarm', rasterize=True, dynamic=False, coastline=True,
    frame_width=240)

temp_globe = temp_ds.sst.interactive.sel(time=pn.widgets.DiscreteSlider).hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True, 
    cmap='jet', rasterize=True, dynamic=False, coastline=True,
    frame_width=240)

In [11]:
plot_measures_button.width_policy = 'max'
year_slider_start_comb.width_policy = 'max'
year_slider_end_comb.width_policy = 'max'

plot_anom_button.width_policy = 'max'
year_slider_start_anom.width_policy = 'max'
year_slider_end_anom.width_policy = 'max'


# Mapper

In [12]:
def years_classification():

    el_nino_years = [1897, 1900, 1903, 1906, 1915, 1919, 1926, 1931, 1941, 1942, 1958, 1966, 1973, 1978, 1980, 1983, 1987, 1988, 1992, 1995, 1998, 2003, 2007, 2010, 2016]

    neutral_ranges = ['1896', '1898-1899', '1901-1902', '1905', '1907-1908', '1912-1914', '1916', '1920-1924', '1927-1930', '1932-1933', '1935-1938', '1940', '1944-1949', '1952-1954', '1957', '1959-1961', '1963-1965', '1967-1970', '1972', '1975', '1977', '1979', '1981-1982', '1984-1986', '1990-1991', '1993-1994', '1996-1997', '2001-2002', '2004-2006', '2009', '2013-2015', '2017-2020']

    la_nina_years = [1904, 1909, 1910, 1911, 1917, 1918, 1925, 1934, 1939, 1943, 1950, 1951, 1955, 1956, 1962, 1971, 1974, 1976, 1989, 1999, 2000, 2008, 2011, 2012, 2021, 2022]

    def convert_ranges_to_years(ranges_list):
        years = []
        for item in ranges_list:
            if '-' in item:
                start, end = item.split('-')
                years.extend(range(int(start), int(end) + 1))
            else:
                years.append(int(item))
        return years

    neutral_years = convert_ranges_to_years(neutral_ranges)

    return el_nino_years, neutral_years, la_nina_years

def classify_year(year):
    nino_years, neutral_years, nina_years = years_classification()
    if year in nino_years:
        return 'Niño'
    elif year in nina_years:
        return 'Niña'
    elif year in neutral_years:
        return 'Neutral'
    else:
        return 'Unknown'  # in case there are years not covered by the lists

In [13]:
# Load Data
sst_anom, _ = get_sst_anoms()
sst_data, _ = get_sst_data()
olr_data, _ = get_olr_data()
olr_anom, _ = get_olr_anoms()
soi_data, _ = get_soi_anoms()


In [14]:
sst_anom_m = sst_anom.query('YEAR > 1896')
sst_data_m = sst_data.query('YEAR > 1896')
olr_data_m = olr_data.query('YEAR > 1896')
olr_anom_m = olr_anom.query('YEAR > 1896')
soi_data_m = soi_data.query('YEAR > 1896')


data = [sst_anom_m, sst_data_m, olr_data_m, olr_anom_m, soi_data_m]
names = ['sst_anom', 'sst_data', 'olr_data', 'olr_anom', 'soi_data']

for df, name in zip(data, names):
    variable = df.drop('YEAR', axis=1).to_numpy()
    variable_scaled = variable - np.mean(variable, axis=0) / np.std(variable, axis=0)

    year_avg_variable = np.mean(variable[:, 1:], axis=1)
    mapper = km.KeplerMapper(verbose=1)

    projected_data = mapper.fit_transform(variable_scaled, 
                                          projection=[manifold.Isomap(n_components=6, n_jobs=-1), 
                                                      PCA(2)]) 

    covering = km.Cover(n_cubes=2, perc_overlap=0.2)

    G = mapper.map(projected_data, X=variable_scaled, cover=covering, clusterer=KMeans(n_clusters=2))

    # define a filename
    #fileID = 'projection=' + G['meta_data']['projection'].split('(')[0] + '_' + \
    #'n_cubes=' + str(G['meta_data']['n_cubes']) + '_' + \
    #'perc_overlap=' + str(G['meta_data']['perc_overlap']) + '_' + \
    #'clusterer=' + G['meta_data']['clusterer'].split('(')[0] + '_' + \
    #'scaler=' + G['meta_data']['scaler'].split('(')[0]

    # visualize graph
    #mapper.visualize(G, 
    #                path_html= "mappers/"name + "_mapper_" + fileID + ".html",
    #                title=fileID,
    #                custom_tooltips = df.YEAR.apply(classify_year).to_numpy(),
    #                color_values = year_avg_variable,
    #                color_function_name = 'Average ' + name,
    #                node_color_function = np.array(['average', 'std', 'sum', 'max', 'min']))

    # Dynamically create a global variable
    globals()[f"G_{name}"] = G

KeplerMapper(verbose=1)
..Composing projection pipeline of length 2:
	Projections: Isomap(n_components=6, n_jobs=-1)
		PCA(n_components=2)
	Distance matrices: False
False
	Scalers: MinMaxScaler()
MinMaxScaler()
..Projecting on data shaped (126, 12)

..Projecting data using: 
	Isomap(n_components=6, n_jobs=-1)


..Scaling with: MinMaxScaler()

..Projecting on data shaped (126, 6)

..Projecting data using: 
	PCA(n_components=2)


..Scaling with: MinMaxScaler()

Mapping on data shaped (126, 12) using lens shaped (126, 2)

Creating 4 hypercubes.

Created 8 edges and 8 nodes in 0:00:00.034416.
KeplerMapper(verbose=1)
..Composing projection pipeline of length 2:
	Projections: Isomap(n_components=6, n_jobs=-1)
		PCA(n_components=2)
	Distance matrices: False
False
	Scalers: MinMaxScaler()
MinMaxScaler()
..Projecting on data shaped (73, 12)

..Projecting data using: 
	Isomap(n_components=6, n_jobs=-1)


..Scaling with: MinMaxScaler()

..Projecting on data shaped (73, 6)

..Projecting data using

In [15]:
import panel as pn
import hvplot.pandas
from PIL import Image

# Define RadioButtonGroup widget for graph selection
graph_button = pn.widgets.RadioButtonGroup(
    name='Graph', 
    button_type = 'primary',
    options=['G_sst_anom', 'G_sst_data', 'G_olr_data', 'G_olr_anom', 'G_soi_data'], 

)

# Define a Select widget with cluster options
cluster_select = pn.widgets.Select(
    name='Select Cluster', 
    options=['cube0_cluster0', 'cube0_cluster1', 'cube1_cluster0', 'cube1_cluster1',
             'cube2_cluster0', 'cube2_cluster1', 'cube3_cluster0', 'cube3_cluster1']
)

# Define a Toggle widget to switch image paths
with_button = pn.widgets.Toggle(name='Label ON/OFF', button_type='primary')

# Define a function to create the DataFrame based on the selected cluster and graph
@pn.depends(graph_button.param.value, cluster_select.param.value)
def create_df(graph, cluster):
    G = globals()[graph]  # Get the selected Graph
    df = globals()[graph[2:]+'_m']
    dfnode = df.iloc[G['nodes'][cluster], :]
    dfnode = dfnode.melt(id_vars='YEAR', var_name='MONTH', value_name='TEMPERATURE')
    return dfnode

# Define a function to create the plot based on the DataFrame
@pn.depends(graph_button.param.value, cluster_select.param.value)
def create_plot(graph, cluster):
    dfnode = create_df(graph, cluster)
    return dfnode.hvplot.line(x='MONTH', y='TEMPERATURE', by='YEAR', xlabel='Month', ylabel='Temperature', legend='top_right', width = 1250)

# Define a function to display the DataFrame
@pn.depends(graph_button.param.value, cluster_select.param.value)
def display_df(graph, cluster):
    dfnode = create_df(graph, cluster)
    dfnode.columns = dfnode.columns.str.strip()
    dfnode = dfnode.reset_index()
    pivot_df = dfnode.pivot(index='YEAR', columns='MONTH', values='TEMPERATURE')
    
    ordered_columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG','SEP', 'OCT', 'NOV', 'DEC']
    pivot_df = pivot_df[ordered_columns]
    
    pivot_df.columns.name = None
    
    df_reset = pivot_df.reset_index().rename(columns={"index": "YEAR"})
    df_reset['TYPE'] = df_reset['YEAR'].apply(classify_year)

    df_reset['YEAR'] = df_reset['YEAR'].astype(str)
    df_table = df_reset.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
    return df_table


cluster_select.width_policy = 'max'
graph_button.width_policy = 'max'
with_button.width_policy = 'max'

# Function to display the PNG image with adjustable anti-aliasing
def display_image(graph, labeled, width=None, height=None, resample_filter=Image.LANCZOS):
    if labeled:
        image_path = f"images/{graph}_labeled.png"
    else:
        image_path = f"images/{graph}.png"
    img = Image.open(image_path)
    img = img.resize((width, height), resample=resample_filter)
    img_widget = pn.panel(img)
    return img_widget

# Define a function to update the image display based on the selected graph
@pn.depends(graph_button.param.value, with_button.param.value)
def update_image_display(graph, with_labeled):
    return display_image(graph, with_labeled, width=450, height=400, resample_filter=Image.BICUBIC)  # Adjust the filter as needed


# Persistent Homology for ENSO Analysis

In [16]:
# GET DATA

data = get_merged_data().dropna()

test_data = data.query('DATE >= "1985"')
test_data = test_data.drop(columns = ['TEMP', 'OLR'])

In [17]:
# ANOM AND TEMPERATURES
P = test_data.drop(columns = ['DATE']).to_numpy() + 10
# ONLY ANOM
# P = nino34_anom[['ANOM']].to_numpy() + 5
# ONLY TEMPERATURES
# P = nino34_anom[['TEMP']].to_numpy() 
r = np.log(np.divide(P[1:],P[:len(P)-1]))

In [18]:
# Instantiate Vietoris-Rips solver
rips = Rips(maxdim = 2)


dgm = rips.fit_transform(r[0:100])

#plt.figure(figsize=(5, 5), dpi=80)
#plt.rcParams.update({'font.size': 10})
#persim.plot_diagrams(dgm, title="Persistence Diagram")


# Save the figure as a png file
# plt.savefig('persistenceDiagram.png')

# plt.close()

Rips(maxdim=2, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [19]:

def visualize_persistent_homology(X ,max_dimension = 2):
    
    D = pairwise_distances(X)
    skeleton = gd.RipsComplex(distance_matrix = D, max_edge_length = 10) 
    Rips_complex = skeleton.create_simplex_tree(max_dimension = max_dimension)
    BarCodes = Rips_complex.persistence()

    for dim in range(max_dimension):
        print('Dimension',dim)
        plt.figure()
        gd.plot_persistence_barcode([bar for bar in BarCodes if bar[0] == dim])
        plt.savefig(f'persistenceBarCode_dim{dim}.png')
        plt.show()
            
    return

#visualize_persistent_homology(P)

In [20]:
# Instantiate Vietoris-Rips solver
rips = Rips(maxdim = 2)

# some parameters
w = 60 # time window size
n = len(test_data)-(2*w)+1 # number of time segments
wasserstein_dists = np.zeros((n,1)) # initialize array for wasserstein distances

# compute wasserstein distances between persistence diagrams for subsequent time windows
for i in range(n):
    # Compute persistence diagrams for adjacent time windows
    dgm1 = rips.fit_transform(r[i:i+w])
    dgm2 = rips.fit_transform(r[i+w+1:i+(2*w)+1])
    
    # Compute wasserstein distance between diagrams
    wasserstein_dists[i] = persim.wasserstein(dgm1[0], dgm2[0], matching=False)

Rips(maxdim=2, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [21]:

# Supposing that 'test_data' and 'wasserstein_dists' are defined
df = pd.DataFrame({
    'Date': test_data['DATE'][w+1:n+w+1],
    'Wasserstein': wasserstein_dists.reshape(-1),
    'SST ANOM Scaled': test_data['SST ANOM'][w+1:n+w+1]/10+1.3
})



# Convert dates to integers (Unix timestamp)

wasserstein_plot = df.hvplot(x='Date', y='Wasserstein', width=1250, height=400, label='Wasserstein',shared_axes=False).opts(fontsize='7.5pt')
sst_anom_plot = df.hvplot(x='Date', y='SST ANOM Scaled', width=1250, height=400, label='SST ANOM Scaled',shared_axes=False).opts(fontsize='7.5pt')

persistence_plot = wasserstein_plot * sst_anom_plot

# Set the title of the persistence_plot
persistence_plot = persistence_plot.opts(title='Can Homology Changes Predict ENSO Events?')

# PERIODICITY

In [22]:
stride = 7

# 0 - connected components, 1 - loops, 2 - voids
homology_dimensions = [0, 1, 2]

periodic_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)

plots_dict = {}

def embed(points, window=2):
    auxembed = []
    for i in range(0, len(points) - window + 1):
        aux = [points[i + j] for j in range(window)]
        auxembed.append(aux)
    auxembed = pd.DataFrame(auxembed, columns=['x', 'y'])
    return auxembed

def update_dataframe(column):
    y = comb[column].dropna().to_numpy()
    y_embedded_2d = embed(y)
    max_periodic_dimension = round(len(y) ** (1 / 2))
    max_periodic_time_delay = round(len(y) ** (1 / 2))
    tau, d = takens_embedding_optimal_parameters(y, max_periodic_dimension, max_periodic_time_delay, stride)

    embedder_periodic = SingleTakensEmbedding(
        parameters_type="search",
        n_jobs=2,
        time_delay=tau,
        dimension=d,
        stride=stride,
    )

    y_embedded = embedder_periodic.fit_transform(y)

    pca = PCA(n_components=3)
    y_embedded_pca = pca.fit_transform(y_embedded)

    y_embedded_pca1 = y_embedded_pca[None, :, :]
    diagrams = periodic_persistence.fit_transform(y_embedded_pca1)

    return y_embedded_2d, y_embedded_pca, diagrams



for column in comb.columns[1:-1]:
    y_embedded_2d, y_embedded_pca, diagrams = update_dataframe(column)
    plot1 = plot_point_cloud(y_embedded_pca)
    plot2 = plot_diagram(diagrams[0])
    
    line_plot = y_embedded_2d.hvplot.line(x='x', y='y', line_width=1.5, color='SlateGray', width=1250, height=350)
    scatter_plot = y_embedded_2d.hvplot.scatter(x='x', y='y', marker='o', size=30, color='SlateGray', width=1250, height=350)

    plot3 = line_plot * scatter_plot
    
    plots_dict[column] = plot1, plot2, plot3
        

def load_image(filename):
    img = Image.open(filename)
    return pn.pane.PNG(img)



periodicity_col = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    button_type = 'primary',
    options= list(comb.columns[1:-1])
)







In [23]:
@pn.depends(periodicity_col=periodicity_col.param.value)
def periodicity_plot_2d(periodicity_col):
    return plots_dict[periodicity_col][2]
    
@pn.depends(periodicity_col=periodicity_col.param.value)
def periodicity_cloud_point(periodicity_col):
    return plots_dict[periodicity_col][0]

@pn.depends(periodicity_col=periodicity_col.param.value)
def periodicity_diagram(periodicity_col):
    return plots_dict[periodicity_col][1]

periodicity_col.width_policy = 'max'


# Forecasting

In [24]:
# X, y = get_forecasting_data(tda = True)

# Suppose that 'y' is your target series
# y_train = y[:-171]
# y_test = y[-171:]

# Splitting data into training and testing sets
# X_train = X.sel(time=slice('1880-01-01', '2009-01-01'))
# X_test = X.sel(time=slice('2009-01-01', '2023-03-01'))

In [25]:
# #Takens Embedding
# embedding_dimension = 5  # Number of lags
# embedding_time_delay = 2  # Separation between lags

# embedder = TakensEmbedding(dimension=embedding_dimension, time_delay=embedding_time_delay)

## Fitting and transforming the training data
# embedder.fit(X.values)
# X_train_embedded = embedder.transform(X_train.values)

## Transforming the test data
# X_test_embedded = embedder.transform(X_test.values)

In [26]:
# Define the pipeline
VR = VietorisRipsPersistence(metric='euclidean', homology_dimensions=[0, 1])
Ampl = Amplitude(metric='landscape', order=None)
RF = RandomForestRegressor()

In [27]:
tda_pipe = make_pipeline(VR, Ampl, RF)
# pipe.fit(X_train_embedded, y_train)

In [28]:
## Make predictions
#y_pred = pipe.predict(X_test_embedded)

## Calculate the score
#rmse = mean_squared_error(y_test, y_pred, squared=False)
#print('RMSE:', rmse)
#score = pipe.score(X_test_embedded, y_test)
#print('SCORE:', score)

In [29]:
import hvplot.pandas  # noqa

## Create a DataFrame for easier plotting
# df_tda_forceasting = pd.DataFrame({
#    "Original Data": y_test,
#    "Predictions": y_pred
#}, index=y_test.index)

# df_tda_forceasting.to_csv('data/df_tda_forecasting.csv', index=True)

df_tda_forceasting = pd.read_csv('data/df_tda_forecasting.csv', index_col=0)

# Create the hvplot
tda_plot = df_tda_forceasting.hvplot.line(width=1250, height=350).opts(xformatter=formatter)

In [32]:
X, y = get_forecasting_data()


# Del 1980-01-01 hasta 1995-12-01
X_train = X[1200:1392]
y_train = y[1200:1392]

# Del 1996-01-01 al 2006-12-31
X_test = X[1392:1548]
y_test = y[1392:1548]

# Del 2007-01-01 hasta 2018-01-01
X_val = X[-171:]
y_val = y[-171:]

In [36]:
# Create Xgboost model

reg = xgb.XGBRegressor(n_estimators = 1000, 
                       early_stopping_rounds = 50,
                       )
#reg.fit(X_train, y_train, 
#        eval_set = [(X_train, y_train), (X_test, y_test)],
#        verbose = False
#        )

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [37]:
#predictions = reg.predict(X_val)
#corr, _ = scipy.stats.pearsonr(predictions, y_val)
#rmse = mean_squared_error(y_val, predictions)
#print("RMSE: {:.2f}".format(rmse))
#print(f'Score: {reg.score(X_test, y_test)}')

RMSE: 0.25
Score: 0.9101009600757152


In [38]:
import plotly.graph_objects as go


## Create a DataFrame for easier plotting
# df_xg_forecasting = pd.DataFrame({
#    "Original Data": y_val,
#    "Predictions": predictions
#}, index=y_val.index)

# df_xg_forecasting.to_csv('data/df_xg_forecasting.csv', index=True)

df_xg_forecasting = pd.read_csv('data/df_xg_forecasting.csv', index_col=0)


# Create the hvplot
xgb_plot = df_xg_forecasting.hvplot.line(width=1250, height=350).opts(xformatter=formatter)


# DASHBOARD

In [39]:
template = pn.template.FastListTemplate(
    
    title='SST Anomalies Dashboard', 
    
    sidebar=[pn.pane.Markdown("# Sea Surface Temperatures and Anomalies "), 
             pn.pane.Markdown("#### El Niño/ Southern Oscillation (ENSO) is the dominant mode of variability that affects the climate on seasonal time scales.  It is measured by the Nino3.4 index, a rolling 3-month average of equatorial Pacific temperatures.  ENSO is an oscillation and is marked by two phases: El Niño, with anomalously warm equatorial Pacific temperatures, and La Niña, with anomalously cold temperatures.  Because El Niño is tied to many weather patterns around the world, such as the Indian monsoon, hurricanes in the Atlantic, and North American temperature, accurate ENSO forecasts are valuable for climate-sensitive sectors (such as agriculture, water, and energy)."), 
             pn.pane.Markdown(""),
             pn.pane.Markdown(""),
             pn.pane.Markdown(""),
             pn.pane.Markdown(""), 
             temp_globe,
             anom_globe
            ],
    
    main=[
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## Measures Time Series"),
                plot_measures_button,
                year_slider_start_comb, 
                year_slider_end_comb,
                measures_plot.panel(width=1250), 
                margin=(0,25)
            )  
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## Anomalies Time Series"),
                plot_anom_button,
                year_slider_start_anom,
                year_slider_end_anom,
                anom_plot.panel(width=1250), 
                margin=(0,25)
            ) 
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## SST Anomalies with Time Series"),
                pn.pane.Video('media/complete_anomalies.mp4', width=1250, height=800)
            )
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## Periodicity of ENSO Variables"),
                periodicity_col,
                pn.pane.Markdown("## 2D Embedded Data"),
                periodicity_plot_2d
            )
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## Cloud Point Embedded Data "),
                periodicity_cloud_point
            ),
            pn.Column(
                pn.pane.Markdown("## Percistence Diagram "),
                periodicity_diagram
            ),
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## The Mapper Algorithm for ENSO"),
                pn.pane.Markdown('Inspired by : https://github.com/ShawhinT'),
                graph_button, 
                pn.pane.Markdown("### Cluster Graph"),
                create_plot
            ),
        ),
        pn.Row(
            pn.Column(
                pn.Row(
                    pn.pane.Markdown("### Mapper"), with_button
                ),
                update_image_display
            ),
            pn.Column(
                cluster_select,
                pn.pane.Markdown("### Cluster Table"),
                display_df
            )
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## Persistent Homology for ENSO Analysis"),
                pn.pane.Markdown('Inspired by : https://arxiv.org/abs/1703.04385'),
                persistence_plot
            )
        ),
        pn.Row(

            pn.Column(
                pn.Row(
                        pn.pane.Markdown("## SST Anomalies Data for Forecasting"),
                ),
                pn.pane.Markdown("### Own Anomalies Geographic Projection"),
                pn.pane.GIF('media/own_anomalies.gif', width=625, height=500)
            ),
            pn.Column(
                pn.Row(
                    pn.pane.Markdown("## ")
                ),
                pn.pane.Markdown("### NERSC Anomalies Geographic Projection"),
                pn.pane.GIF('media/nersc_anomalies.gif', width=625, height=500)
            )                                
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("### Topology in ENSO time series forecasting (Score = -0.21)"),
                pn.pane.Markdown("Inspired by: https://giotto-ai.github.io/gtda-docs/latest/notebooks/time_series_forecasting.html"),
                tda_plot
            )
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("### XGBoost for ENSO time series forecasting (Score = 0.91)"),
                xgb_plot
            )
        )
        ],
    
    accent_base_color="#2E86C1",
    header_background="#2E86C1",
)
template.show()
template.servable();

Launching server at http://localhost:51992
